In [ ]:
''' Logistic Regression '''

#calculating sigoid function g(z)=1/(1+e^(-z)

def logistic_predicted(x,w):
  return 1/(1+ pow(math.e,-(np.dot(x,w))))

#updating co-eff (wj)
def logistic_w_update(x,y,alpha,w):

  for j in range(len(w)):
    subtract_part = 0
    for i in range(len(x)):
      subtract_part = subtract_part + (logistic_predicted(x[i],w)-y[i])*x[i][j]*(logistic_predicted(x[i],w))*(1-logistic_predicted(x[i],w))

    w[j] = w[j] - (alpha*subtract_part)/len(x)

  return w
#cost function
def error_logistic(x,y,w):
  J = 0


  for i in range(len(x)):
    J = J + (y[i]*math.log(logistic_predicted(x[i],w),10)) + ((1-y[i])*math.log(1-logistic_predicted(x[i],w),10))
  J = -J/(len(x))

  return J

def logloss(df,w,alpha,rho,epochs,showPlot):

  a = np.array(df.values)
  x = a[:,0:len(a[0])-1]
  y = a[:,len(a[0])-1]

  J = 0
  prevJ = 0
  itr = 0
  while True:

    if(itr >= epochs):
      break


    h = []


    for i in range(len(x)):

      # predicted value must be changed
      predicted_value = logistic_predicted(x[i],w)
      # predicted_value = logistic_predicted(x[i],w)
      h.append(predicted_value)

      # gradient
      # J = J + pow(predicted_value - y[i],2)

      # stocastic - logloss
      J = J + (y[i]*math.log(predicted_value,10)) + ((1-y[i])*math.log(1-predicted_value,10))

      for j in range(len(w)):

         # w[j] = w[j] - alpha*(predicted_value-y[i])*x[i][j]
        # logloss-logistic, gradient-stocastic - MSE
        # logistic - MSE
         w[j] = w[j] - alpha*(predicted_value-y[i])*x[i][j]*predicted_value*(1-predicted_value)

    # J = J/(2*len(x)) for batch grad
    J = -J/(len(x)) #for stochastic

    if(abs(J-prevJ) <= rho):
      break

    prevJ = J
    itr = itr + 1

  return J,w

In [ ]:
import pandas as pd
import random
import math
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


df=pd.read_csv("Dataset.csv")
print(df)

#Normalisation of data frame

a=df['Target']
print(a)
df=df.drop(['Target'],axis=1)
df = df/(df.max().max())
df.insert(14,'Target',a)
df.insert(loc=0,column='-1',value=[1 for i in range(len(df))])
print(df)
data_train,data_remaining = train_test_split(df,train_size = 0.3)
data_test,data_validation = train_test_split(data_remaining,train_size = 0.28)
#print(data_train)
#print(data_validation)
c={}
for i in range(len(df['Target'])):
  c=set(df['Target'])
print(c)

alphaarr = np.array([0.1,0.001,0.02,0.03,0.001])
rhoarr = np.array([0.0001,0.0002,0.0003,0.0004,0.0005])
epocharr = np.array([10,20,30,50,80])
W=[]
for i in range(df.shape[1]-1):
     W.append(random.uniform(-0.3,0.3))
print(W)
print(len(W))   
freq={}
for i in c:
  validation_copy=data_validation.copy()
  #print(validation_copy['Target'])
  for j in validation_copy.index :
    if validation_copy['Target'][j] == i:
       validation_copy['Target'][j] = 1
    else:
       validation_copy['Target'][j] = 0
  #print(validation_copy)
  #print(np.array(validation_copy['Target']))
  mse = []
  for j in range(len(alphaarr)):
    error,z=(logloss(validation_copy, W, alphaarr[j], rhoarr[j], epocharr[j],1))
    mse.append(error)
  mini = math.inf
  tmp = 0
  for j in range(0, len(mse)):
    if mse[j] < mini:
      mini = mse[j]
      tmp = j
  if (tmp in freq):
    freq[tmp] += 1
  else:
    freq[tmp] = 1     
v = list(freq.values())
ke = list(freq.keys())
bestIndex = ke[v.index(max(v))]
print("The best hyperparameters are :")
print(alphaarr[bestIndex])
print(rhoarr[bestIndex])
print(epocharr[bestIndex])
print(bestIndex)
#plt.plot(epocharr,mse)
#plt.xlabel("EPOCHS")
#naming the y axis
#plt.ylabel("MSE")
#plt.show()
  

      xF   yF   wF   hF  xRE  yRE  xLE  ...   xN   yN  xRM  yRM  xLM  yLM  Target
0    292  209  100  112  323  232  367  ...  353  254  332  278  361  278       2
1    286  200  109  128  324  235  366  ...  353  258  333  281  361  281       2
2    290  204  105  121  325  240  367  ...  351  260  334  282  362  282       2
3    287  202  112  118  325  230  369  ...  353  253  335  274  362  275       2
4    290  193  104  119  325  224  366  ...  353  244  333  268  363  268       2
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...     ...
601  278  183  128  141  307  218  354  ...  330  247  324  273  356  266       2
602  268  186  128  134  296  222  344  ...  319  247  316  274  347  269       1
603  264  187  127  131  287  220  334  ...  304  247  305  272  337  270       1
604  264  175  143  136  295  207  345  ...  320  234  314  261  351  251       2
605  266  170  141  139  303  206  354  ...  331  229  319  255  362  247       2

[606 rows x 15 

In [ ]:
'''
TRAINING + OVERFITTING - STARTED
'''

def trainAndOverFit(classes,w,train_set,validation_set,bestHyperParameterIndex):
  ClassW = []
  for c in classes:
    trainCopy = train_set.copy(deep=True)
    trainCopyOutput = list(trainCopy['Target'])

    for i in range(len(trainCopyOutput)):
      if(trainCopyOutput[i]==c):
        trainCopyOutput[i] = 1
      else:
        trainCopyOutput[i] = 0

    trainCopy['Target'] = trainCopyOutput
    error,returnedW = logloss(trainCopy,w.copy(),alphaarr[bestHyperParameterIndex],rhoarr[bestHyperParameterIndex],epocharr[bestHyperParameterIndex],'train - class - '+str(c))
    # print(error)

    ClassW.append(returnedW)

    validCopy = validation_set.copy(deep=True)

    validCopyOutput = list(validCopy['Target'])

    for i in range(len(validCopyOutput)):
      if(validCopyOutput[i]==c):
        validCopyOutput[i] = 1
      else:
        validCopyOutput[i] = 0

    validCopy['Target'] = validCopyOutput
    returnedW,error = logloss(validCopy,w.copy(),alphaarr[bestHyperParameterIndex],rhoarr[bestHyperParameterIndex],epocharr[bestHyperParameterIndex],'validation - class - '+str(c))
    # print(error)

  return ClassW

In [ ]:
from sklearn import metrics
def logg_predicted(w,x):
  return 1/(1+ pow(math.e,-(np.dot(w,x))))

def findAccuracy(classes,df,everyClassW):
  a = np.array(df.values)
  x = a[:,0:len(a[0])-1]
  y = a[:,len(a[0])-1]




  y_predicted=[]
  correctlyPredicted=0
  for i in range(len(a)):
    predictedForIthPattern = []
    for c in range(3):
      predictedForIthPattern.append(logg_predicted(x[i],everyClassW[c]))
      
    #print(predictedForIthPattern)

    # predicting the class.
    y_predicted.append(predictedForIthPattern.index(max(predictedForIthPattern))+1)
    #print(predictedClass)
    
    if(y_predicted[i] == y[i]):
      correctlyPredicted += 1
  print('Correctly Predicted : ',correctlyPredicted)
  print('Total Test Samples : ', len(a))
  #print('Accuracy : ',(correctlyPredicted*100)/len(a))
  print()
  
  cnf_matrix = metrics.confusion_matrix(y, y_predicted)
  print("confusion Matrix=",cnf_matrix)
   
  



  for i in range(len(cnf_matrix)):
    print('precision for ',i,' class : ',cnf_matrix[i][i]/(sum(cnf_matrix[i])))

  print()

  colWiseSum_ConfusionMatrix = np.sum(cnf_matrix,axis=0)
  for i in range(len(colWiseSum_ConfusionMatrix)):
    print('recall for ',i,' class : ',cnf_matrix[i][i]/(colWiseSum_ConfusionMatrix[i]))

  return (correctlyPredicted*100)/len(a)




In [ ]:
everyClassW = trainAndOverFit(c,W,data_train.copy(),data_validation.copy(),bestIndex)
print(everyClassW)
print("****TRAIN - ACCURACY*****\n")
print("Accuracy",findAccuracy(c,data_train.copy(),everyClassW))

print()

print("****TEST - ACCURACY*****\n")
print("Accuracy",findAccuracy(c,data_test.copy(),everyClassW))

print()

# '''

print()

[[-0.4269240240749165, -0.22427639468746113, -0.04402949171287178, -0.32107126691715765, -0.005060381589146403, -0.43924312765258533, -0.3335302405759571, -0.4920606617943959, -0.290254233807302, -0.1894515886309163, -0.1594789587915911, -0.34102160717273283, -0.3745517795797807, -0.3697231340059567, -0.579664687950377], [0.2554844451228581, 0.23363692663097602, 0.27478671360352197, -0.14815032692645203, 0.18896379680743927, 0.07629665074913132, 0.025480750640927387, 0.09618858128499144, 0.07066984569762834, 0.3702247258420809, 0.23625178304689784, 0.1861495068546282, 0.06126893863776617, 0.20915734860427704, -0.14277745520971014], [-0.4267976721259471, -0.22405483537655368, -0.043984294614244024, -0.3210428770331537, -0.005013193572924405, -0.4389576032591118, -0.3334835298648275, -0.49178544488317455, -0.29019330025025597, -0.18910570675148441, -0.1594226556501513, -0.340744559085697, -0.37449034003472986, -0.36945935211644415, -0.5795881286056925]]
****TRAIN - ACCURACY*****

Correct